# Llamav2を試す

# 参考
[Databricksレポジトリ](https://github.com/databricks/databricks-ml-examples/tree/master/llm-models/llamav2)  
[Qiita: MetaのLlama 2をDatabricksで動かしてみる](https://qiita.com/taka_yayoi/items/bc1da42144826da56ab4)

# modelのダウンロード

In [3]:
# Load model to text generation pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# it is suggested to pin the revision commit hash and not change it for reproducibility because the uploader might change the model afterwards; you can find the commmit history of llamav2-7b-chat in https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/commits/main
model = "meta-llama/Llama-2-7b-chat-hf"
revision = "0ede8dd71e923db6258295621d817ca8714516d4"

tokenizer = AutoTokenizer.from_pretrained(model, padding_side="left")
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    revision=revision,
    return_full_text=False
)

# Required tokenizer setting for batch inference
pipeline.tokenizer.pad_token_id = tokenizer.eos_token_id


c:\Users\noda7\miniconda3\envs\llamav2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.01s/it]
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


# プロンプトの準備

In [4]:
# Define prompt template, the format below is from: http://fastml.com/how-to-train-your-own-chatgpt-alpaca-style-part-one/

# Prompt templates as follows could guide the model to follow instructions and respond to the input, and empirically it turns out to make Falcon models produce better responses

INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)


# 関数の作成

In [5]:
# Define parameters to generate text
def gen_text(prompts, use_template=False, **kwargs):
    if use_template:
        full_prompts = [
            PROMPT_FOR_GENERATION_FORMAT.format(instruction=prompt)
            for prompt in prompts
        ]
    else:
        full_prompts = prompts

    if "batch_size" not in kwargs:
        kwargs["batch_size"] = 1
    
    # the default max length is pretty small (20), which would cut the generated output in the middle, so it's necessary to increase the threshold to the complete response
    if "max_new_tokens" not in kwargs:
        kwargs["max_new_tokens"] = 2024

    # configure other text generation arguments, see common configurable args here: https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.GenerationConfig
    kwargs.update(
        {
            "pad_token_id": tokenizer.eos_token_id,  # Hugging Face sets pad_token_id to eos_token_id by default; setting here to not see redundant message
            "eos_token_id": tokenizer.eos_token_id,
        }
    )

    outputs = pipeline(full_prompts, **kwargs)
    outputs = [out[0]["generated_text"] for out in outputs]

    return outputs


# 単一の入力で文書生成。

In [4]:
results = gen_text(["What is a large language model?"])
print(results[0])

c:\Users\noda7\miniconda3\envs\llamav2\lib\site-packages\transformers\generation\utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(



A large language model is a type of artificial intelligence (AI) model that is trained on a large corpus of text data to generate language outputs that are coherent and natural-sounding. These models are designed to learn the patterns and structures of language by exposure to a wide range of texts, and can be used for a variety of applications such as language translation, text summarization, and language generation.

Some examples of large language models include:

1. BERT (Bidirectional Encoder Representations from Transformers): Developed by Google in 2018, BERT is a powerful language model that has achieved state-of-the-art results on a wide range of natural language processing (NLP) tasks. BERT uses a multi-layer bidirectional transformer encoder to generate contextualized representations of words in a sentence.
2. RoBERTa (Robustly Optimized BERT Pretraining Approach): Developed in 2019, RoBERTa is a variant of BERT that was specifically designed for text classification tasks. R

# 日本語は？

In [6]:
results = gen_text(["大規模言語モデルとは？"])
print(results[0])



大規模言語モデル（Big Language Model、BLM）とは、自然言語processing（NLP）における一種の人工知能（AI）で、大規模なデータセットを用いて、自然言語の表現を処理することができるように設計されている。

BLMは、通常、大量の文書やデータセットを用いて、言語モデルを学習することができる。これらのデータセットには、単語や文、文章などの自然言語の表現が含まれている。BLMは、これらのデータセットを用いて、自然言語の表現を処理するための特殊な neural network を学習する。

BLMは、以下のような役割を果たすことができる。

1. 文本生成：BLMを使用すると、新しい文本を生成することができます。これには、単語や文、文章などの自然言語の表現が含まれています。
2. 文本理解：BLMを使用すると、既存の文本を理解することができます。これには、単語や文、文章などの自然言語の表現が含まれています。
3. 単語の定義：BLMを使用すると、単語の定義を学習することができます。これには、単語の意味や、単語が使用される文脈などが含まれています。
4. 文法分析：BLMを使


In [8]:
results = gen_text(["""桃太郎の物語を教えてください。                   
"""])
print(results[0])


桃太郎は、日本の民話の登場人物であり、「桃太郎」という名前を持つ。彼は、桃の木の下で生まれ、桃太郎と名付けられた。彼は、妖精の姉妹たちとともに暮らし、桃の木の下で暮らしていた。

ある日、桃太郎は、妖精たちとともに桃の木の下で遊んでいた。彼らは、桃の木の下で遊んでいるうちに、桃太郎が妖精たちに対して、「桃太郎の物語」を語り始めた。

彼らは、桃太郎が妖精たちに語った物語を、「桃太郎の物語」と呼んだ。彼らは、桃太郎の物語を聞いて、彼の勇敢さと優しさに感動した。

桃太郎の物語は、妖精たちにとっては、大切な物語であり、彼らの心に深く刻まれた。彼らは、桃太郎の物語を、彼の勇敢さと優しさを讃えるために、彼の名前を呼び続けた。

以上が、桃太郎の物語の要点です。彼の物語は、勇敢さと優しさを讃えるために、妖精たちにとっては、大切な物語であり、彼らの心に深く刻まれた。


## 桃太郎の物語を知らないようだった。

# 桃太郎の物語でfine-tuningを行う。